# PP-LiteSeg (A Superior Real-Time Semantic Segmentation Model)  
- 논문 리뷰

## Abstract  
- real-time semantic segmentation task를 위해 새로운 모델 PP-LiteSeg를 제안한다  
- 또한 computation cost를 줄이기 위해 FLD(Flexible and Lightweight Decoder)를 제안한다  
- feature representation을 강화하기 위해 UAFM(Unified Attention Fusion Module)을 제안한다  
- 또한 SPMM(Simple Pyramid Pooling Module)을 제안하며 이는 global context와 low computation cost를 합치는 방식이다  
- 우리의 모델은 Cityscapes test set에서 72% mIoU, 273.6 FPS 그리고 77.5% mIoU, 102.6 FPS를 달성했다  

<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/figure1.JPG?raw=true">

## Introduction  
- FCN은 semantic segmentation에 많이 쓰이는 encoder-decoder architecture를 제시함  
- 그러나 computation cost가 많아 real-time task에 적합하지 않음  
- 그래서 inference speed를 가속화하기 위해 다양한 method들이 제시됨  
- Espnet v2는 large receptive field를 얻기 위해 lightweight convolution을 활용  
- BiSeNetV2는 detail feature와 semantic feature를 따로 추출  
- STDCSeg는 computation 효율을 위해 STDC라는 새로운 backbone을 제안  
- 그러나 이 모델들은 speed와 accuracy 사이의 trade off를 극복하지 못함  
- PP-LiteSeg는 encoder-decoder 구조로 이루어져 있으며 3가지 새로운 module로 구성되어 있음   
    - FLD  
        - 채널수를 서서히 늘리며 spatial size를 줄임  
        - flexible하고 lightweight한 decoder  
        - encoder에 따라 decoder의 volume을 쉽게 조정할 수 있음  
        - computation cost를 줄여주며 효율적인 구조  
        
    - UAFM  
        - decoder에서 low level feature와 high level feature를 결합  
        - 기존의 fusion module은 computation cost가 많음  
        - 그래서 효율적으로 결합하는 UAFM을 제안함  
        
    - SPPM  
        - PSPNet에서 제안했던 PPM의 구조를 활용  
        - 중간, 마지막 채널을 줄이고 short-cut을 삭제하고 concatenate를 add로 대체

## Proposed Method  
### Flexible and Lightweight Decoder  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/figure3.JPG?raw=true">

- encoder-decoder architecture는 semantic segmentation에서 굉장히 효과적임  
- 보통 encoder는 채널 수는 서서히 증가하고 spatial size는 줄어듦  
- 이 과정에서 feature는 high level에서 low level로 증가함  
- 그러나 최근 lightweight 모델의 decoder는 모든 level에서 feature의 채널 수가 같도록 유지됨  
- shallow stage보다 deep stage에서의 computation cost가 더 많음  
- 따라서 deep stage의 채널 수를 많게 잡고 output layer에 가까울수록 channel 수가 적어지는 구조로 구성  
- 즉, 효율적인 computation cost를 가져옴  

### Unified Attention Fusion Module    
<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/figure4.JPG?raw=true">

- feature representation을 강화하기 위해 channel과 spatial attention을 적용하는 UAFM을 제안  
- UAFM은 weight $\alpha$를 생성하기 위해 attention module을 활용  
- 그리고 Mul과 Add 연산에 의해 input feature와 $\alpha$가 결합됨  
- $F_{high}$는 더 깊은 module의 output이며 $F_{low}$는 이전 stage의 output  
- 이때 이 두 output은 같은 채널 수를 가짐  
- $F_{high}$는 $F_{low}$와 같은 사이즈를 가지기 위해 bilinear interpolation으로 upsample을 수행  
- 그러면 $F_{up}$을 얻게 되며 이와 $F_{low}$를 attention module에 입력하면 weight $\alpha$가 생성됨  
- 그 후에 attention weighted feature를 얻기 위해 $F_{up}$과 $F_{low}$의 elemente wise mul 연산을 수행  
- 마지막으로 attention weighted feature에 대해 element wise addition 수행  
- 위의 과정을 간단히 식으로 정리하면 다음과 같음  
$$F_{up} = Upsample\left(F_{high}\right)$$  
$$\alpha=Attention\left(F_{up}, F_{low}\right)$$  
$$F_{out} = F_{up} \cdot \alpha + F_{low}\cdot \left(1-\alpha\right)$$  

#### Spatial Attention Module  
$$F_{cat} = Concat\left(Mean\left(F_{up}\right), Max\left(F_{up}\right), Mean\left(F_{low}\right), Max\left(F_{low}\right)\right)$$    
$$\alpha = Sigmoid\left(Conv\left(F_{cat}\right)\right)$$   

- input feature에서 어떤 pixel이 중요한지 나타낼 수 있음  
- 단순 mean, max 연산을 수행
- spatial attention module은 computation cost를 줄이기 위해 max operation을 제거하는 등 flexible한 module   

#### Channel Attention Module  
$$F_{cat} = Concat\left(AvgPool\left(F_{up}\right), MaxPool\left(F_{up}\right), AvgPool\left(F_{low}\right), MaxPool\left(F_{low}\right)\right)$$  
$$\alpha=Sigmoid\left(Conv\left(F_{cat}\right)\right)$$  

- input feature에서 어떤 채널이 중요한지 나타낼 수 있다는 점이다  
- Average Pooling과 Max Pooling operation을 수행   


- 본 논문은 spatial attention module을 채택함

### Simple Pyramid Pooling Module    
<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/figure5.JPG?raw=true">  

- SPPM(Simple Pyramid Pooling Module)을 제안한다  
- SPPM은 3개의 GAP 연산을 수행하며 ouptput size는 각각 $1\times 1$, $2\times 2$, $4\times 4$이다  
- 이후 output feature들은 convolution과 upsampling 연산을 수행  
- conv 연산에서 kernel size는 $1\times 1$이며 output channel의 수는 input channel의 수 보다 적다  
- 마지막으로 upsample된 feature를 더하고 refine된 feature를 얻기 위해 conv 연산을 한 번 더 수행  
- 기존 PPM과 비교해 SPPM은 중간과 output channel을 줄이고   
- short-cut을 제거하고 concatenation 대신 addition 연산을 사용함  
- 즉, 더 효율적이며 real-time task에 더 적합  

### Network Architecture   
<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/figure2.JPG?raw=true">


- PP-LiteSeg는 다음 3가지 module로 구성됨  
    - encoder  
        - encoder로는 STDCNet을 사용    
        - 5개의 stage를 가지고 있으며 각 stride는 2를 사용  
        - final feature size는 input image의 $1/32$  
        - pre-trained weight를 사용
        
    - aggregation  
        - SPPM을 적용  
        - SPPM은 encoder의 output을 input으로 사용해 global context 정보를 가지고 있는 feature를 생성  
     
    - decoder  
        - FLD를 적용  
        - 2개의 UAFM과 한개의 segmentation head를 포함  
        - 효율성을 위해 UAFM에 spatial attention module을 적용  
        - segmentation head에서 output channel을 class 수에 맞추기 위해 Conv BN ReLU 연산 수행  
        - 그리고 input size에 맞추기 위해 upsample을 수행  
        - OHEM Cross Entropy Loss 사용  

## Experiments  
#### Datasets and Implementation Details  
- Training Settings  
    - optimizer: SGD momentum 0.9  
    - learning rate scheduler: warm poly learning rate scheduler  

- Cityscapes  
    - train: 2975, valid: 500, test: 1525  
    - resolution: $2048\times 1024$  
    - class: the annotated images have 30 classes and out experiments only use 19 classes for a comparison with other methods  
    - batch size: 16  
    - max iteration: 160000  
    - initial learning rate: 0.005  
    - weight decay: 5e-4
   
- CamVid  
    - train: 367, valid: 101, test: 233  
    - resolution: $960\times 720$  
    - class: the annotated images provide 32 categories, of which the subset of 11 categories are used in out experiments  
    - batch size: 24  
    - max iteration: 1000  
    - initial learning rate: 0.01  
    - weight decay: 1e-4
    
- Data augmentation  
    - random scaling with scale ranges in [0.125, 1.5] (cityscapes), [0.5, 2.5] (camvid)
    - random cropping  
    - random horizontal flipping  
    - random color jittering  
    - normalization  
    
- Inference Settings  
    - cityscapes에서는 $1024\times 512$ or $1536\times 768$로 resize 진행  
    - camvid에서는 resolution 유지  

### Experiments on Cityscapes  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/table1.JPG?raw=true">

- PP-LiteSeg에 encoder network를 두 개로 실험  
- PP-LiteSeg-T의 경우 $512\times 1024$로 사용한 결과 273.6 FPS와 72.0% mIoU   
- PP-LiteSeg-B의 경우 $768\times 1536$로 사용한 결과 78.2% mIoU를 달성  

#### Ablation study  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/table3.JPG?raw=true">
<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/figure6.JPG?raw=true">  

- FLD, SPPM, UAFM을 각각 독립적으로 사용하거나 모두 사용했을 때의 성능 비교  
- 모두 사용하면 speed가 조금 떨어지긴 하지만 mIoU가 가장 높음  

### Experiments on CamVid  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/PP-LiteSeg/figure/table4.JPG?raw=true">  

- PP-LiteSeg-T에서는 222.3 FPS와 73.3%의 mIoU를 달성  
- 또한 기존의 SOTA보다 12.5% 더 빠른 speed를 자랑함  
- PP-LiteSeg-B에서는 154.8 FPS와 75.0%의 mIoU를 달성  
- 본 논문에서 제안한 모델은 inference speed에서 SOTA를 달성함  

## Conclusions  
- 본 논문에서 우리는 새로운 semantic segmentation model을 만드는 것에 집중함  
- FLD는 이전의 decoder보다 더 효율적이며 flexible하고 lightweight하게 설계함  
- UAFM은 feature를 합칠 때 representation이 더 보강되도록 설계함  
- SPPM은 적은 계산 비용으로 global context를 결합할 수 있음  
- PP-LiteSeg는 accuracy와 inference speed 사이의 trade-off를 극복하며 SOTA를 달성